### bilibili使用js动态加载，直接爬虫爬不到，需要在f12控制台报文中找到调用的api。

#### 爬取用户关注数据
以我为原点爬取我的关注和我的粉丝，再依次爬取他们的关注和粉丝
数据保存在userdf 和 followdf 中

### 200615 v1.1 重构代码，重写get为getjson。

In [1]:
import time
import math
import requests
import pandas as pd
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'Referer':"https://www.bilibili.com/"
}
# cookies = {
#     "SESSDATA": "0ccb2e9a%2C1601886871%2C7146f*41"
# }


                
########## 工具类 

# 为dataFrame 添加新行
def addDf(df, data):
    df.loc[df.shape[0],:] = data
    
# 获取json信息。
def getJson(url):
    try:
        r = requests.get(url, headers=headers, timeout=5)
        if r.status_code == 412:
            print("当前时间：",time.localtime().tm_hour," : ",time.localtime().tm_min," 休眠一次")
            time.sleep(5*60)
            return {}
        else:
            return r.json()
    except:
        return {}

def saveData():
    userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",  encoding='utf_8_sig')
    followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",  encoding='utf_8_sig')
    
def roadData():
    userdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",index_col=0, dtype={'uid':int,'follower':int,'level':int,'coins':int,'following':int})
    followdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",index_col=0, dtype='int')
    return userdf, followdf

########## 具体类
        
#获取关注数fi 和粉丝数fe 。用于做列表爬虫。
def getFfNum(mid):
    api = "https://api.bilibili.com/x/relation/stat?vmid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    if 'data' in json:
        fi = json['data']['following']
        fe = json['data']['follower']
        return int(fi), int(fe)
    else:
        return -1, -1
    
def getUserName(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    if 'data' in json:
        return str(json['data']['name'])
    else:
        return 'null'


# 在userdf中添加数据
def addUser(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    fi, fe = getFfNum(mid)
    if 'data' in json and json['data'] and fi != -1 and fe != -1:
        data = json['data']
        addDf(userdf, [data['mid'], data['name'], data['sex'], data['level'], data['birthday'], data['coins'], fi, fe])

# 获取用户mid 关注列表filist
def getFollowingsList(mid):
    if mid not in userdf['uid'].values:
        addUser(mid)
    fi = userdf[userdf['uid']==mid]['following'].values
    if len(fi)<1:
        print('用户',str(mid),'失败')
        return []
    
    flist = []
    # 除20 向上取整
#     page = math.ceil(fi[0]/20)
    # biliapi 只能访问5页限制。
    page = 5
    for p in range(page):
        api = "https://api.bilibili.com/x/relation/followings?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
        json = getJson(api)
        if 'data' in json and json['data'] and 'list' in json['data']:
            for info in json['data']['list']:
                flist.append(int(info['mid']))
                addDf(followdf, [mid, info['mid'] ])
#     print('用户',str(mid),'完成')
    saveData()
    return flist

# 封装后的getfollow
def getFollowings(mid):
    if type(mid) == list:
        print(mid) # 
        flist = []
        isok = 0
        for m in mid:
            f = getFollowingsList(m)
            if not f:
                isok += 1
            flist.extend(f)
        print('搜取数据数量： ',isok,'  数据搜取成功率： ', isok/len(mid))
        return flist
    else :
        return getFollowingsList(mid)

    
def getFollowersList(mid):
    if mid not in userdf['uid'].values:
        addUser(mid)
    fe = userdf[userdf['uid']==mid]['following'].values
    if len(fe)<1:
        return []
    flist = []
#     除20 向上取整
#     page = math.ceil(fe[0]/20)
    # biliapi 只能访问5页限制。
    page = 5
    for p in range(page):
        api = "https://api.bilibili.com/x/relation/followers?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
        json = getJson(api)

        if 'data' in json and json['data'] and 'list' in json['data']:
            for info in json['data']['list']:
                flist.append(int(info['mid']))
    return flist

def getFollowers(mid):
    if type(mid) == list:
        print(mid) # 
        flist = []
        isok = 0
        for m in mid:
            f = getFollowersList(m)
            if not f:
                isok += 1
            flist.extend(f)
        print('搜取数据数量： ',isok,'  数据搜取成功率： ', isok/len(mid))
        return flist
    else :
        return getFollowersList(mid)


In [ ]:
# 获取某用户相关用户
userdf, followdf = roadData()
def main():
    
    my = 58289054
    # 获取用户的关注数据
    list1 = getFollowings(my)
#     list1 = [456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665157, 215910135, 24127511]

    # 获取被关注的人的粉丝
    list2 = getFollowers(list1)
    
    # 去除重复
    list3 = list(set(list2) - set(list1))
    
    # 对粉丝再次获取关注数据
    list4 = getFollowings(list3)
    
    list5 = getFollowers(list4)
    
    list6 = list(set(list5) - set(list4))
    
    getFollowings(list6)
    
main()

[280793434, 416186469, 350001797, 257797153, 102043515, 526294618, 7552204, 178029850, 208259, 479893558, 506277449, 385128992, 384951376, 64840236, 380128550, 24952714, 92609386, 6926237, 427639872, 245895278, 5970160, 5294454, 30895245, 472586604, 517327498, 17223352, 615236, 346796186, 35461776, 493590366, 344509109, 343055119, 88461692, 403943112, 7788379, 7753700, 221648, 344194038, 21837784, 38136734, 217049242, 357451181, 121682346, 204086589, 268819811, 350905839, 437257786, 470647022, 362169745, 927587, 269380821, 386080586, 23640791, 415270918, 7411106, 66391032, 12413745, 40096793, 32847571, 178534406, 41292603, 382038930, 2884629, 16166225, 8188433, 275153040, 8303290, 52074480, 340223423, 370342489, 15967711, 2284412, 27511366, 234557264, 383289740, 103496542, 330325021, 175456549, 13607452, 12333557, 7720791, 105034258, 330881052, 87641747, 26013256, 313861]
搜取数据数量：  0   数据搜取成功率：  0.0
[444203012, 479199236, 622329864, 202113039, 196630, 555974679, 354254873, 290553883, 56

当前时间： 9  :  19  休眠一次
用户 470909150 失败
当前时间： 9  :  24  休眠一次
用户 404586719 失败
当前时间： 9  :  29  休眠一次
用户 425459939 失败
当前时间： 9  :  34  休眠一次
用户 383320297 失败
当前时间： 9  :  39  休眠一次
用户 88441070 失败
当前时间： 9  :  44  休眠一次
用户 294879472 失败
当前时间： 9  :  57  休眠一次
当前时间： 10  :  2  休眠一次
当前时间： 10  :  7  休眠一次
用户 622330657 失败
当前时间： 10  :  12  休眠一次
当前时间： 10  :  17  休眠一次
用户 488178465 失败
当前时间： 10  :  22  休眠一次
用户 278004515 失败
当前时间： 10  :  27  休眠一次
用户 287376170 失败
当前时间： 10  :  41  休眠一次
当前时间： 10  :  46  休眠一次
用户 620201411 失败
当前时间： 10  :  51  休眠一次
用户 509511108 失败
当前时间： 10  :  56  休眠一次
用户 587236807 失败
当前时间： 11  :  1  休眠一次
用户 21169612 失败
当前时间： 11  :  6  休眠一次
用户 501745101 失败
当前时间： 11  :  19  休眠一次
用户 440961051 失败
当前时间： 11  :  24  休眠一次
用户 430311452 失败
当前时间： 11  :  29  休眠一次
用户 399902759 失败
当前时间： 11  :  34  休眠一次
用户 392759341 失败
当前时间： 11  :  39  休眠一次
用户 320571443 失败
当前时间： 11  :  44  休眠一次
用户 35981369 失败
当前时间： 11  :  57  休眠一次
用户 609880733 失败
当前时间： 12  :  2  休眠一次
用户 854686 失败
当前时间： 12  :  7  休眠一次
用户 448236197 失败
当前时间： 12  :  12  休眠

In [3]:
a = [1, 2, 3]
b = [2, 3]
a - b

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [3]:
list2

NameError: name 'list2' is not defined

### v1.0

In [3]:
import time
import math
import requests
import pandas as pd
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'Referer':"https://www.bilibili.com/"
}
# cookies = {
#     "SESSDATA": "0ccb2e9a%2C1601886871%2C7146f*41"
# }

def Get(url):
    try:
        r = requests.get(url, headers=headers)
        return r
    except:
        return 0

#获取followings followers信息
def getFfNum(mid):
    try:
        api = "https://api.bilibili.com/x/relation/stat?vmid="+str(mid)+"&jsonp=jsonp"
        data = Get(api).json()['data']
        fi = data['following']
        fe = data['follower']
        return int(fi), int(fe)
    except:
        return -1, -1

def addDf(df, data):
    df.loc[df.shape[0],:] = data

def getFollowingsList(mid, fnum):
    if fnum <= 0:
        return []
    flist = []
    # 除20 向上取整
    page = math.ceil(fnum/20)
    for p in range(page):
        try:
            api = "https://api.bilibili.com/x/relation/followings?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
#             print(api)
            for info in Get(api).json()['data']['list']:
                flist.append(int(info['mid']))
                addDf(followdf, [mid, info['mid'] ])
        except:
            continue
    return flist

def getFollowersList(mid, fnum):
    if fnum <= 0:
        return []
    flist = []
    # 除20 向上取整
    page = math.ceil(fnum/20)
    for p in range(page):
        try:
            api = "https://api.bilibili.com/x/relation/followers?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
#             print(api)
            for info in Get(api).json()['data']['list']:
                flist.append(int(info['mid']))
        except:
            continue

    return flist

def getUserName(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    try:
        data = Get(api).json()['data']
        return data['name']
    except:
        return 'error_'+str(mid)

def addUserDf(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    try:
        data = Get(api).json()['data']
        fi, fe = getFfNum(mid)
        datalist = [data['mid'], data['name'], data['sex'], data['level'], data['birthday'], data['coins'], fi, fe]
        addDf(userdf, datalist)
        
        # 后面要进行遍历，防止遍历过长，将上限改为值的3/4次幂。 例 20000-> 1684, 200000 -> 9458
#         fmax = 200000
#         if fi > fmax:
#             fi = fmax
#         if fe > fmax:
#             fe = fmax
        return math.ceil(fi**(3/4)), math.ceil(fe**(3/4)),
    except:
        return -1, -1

    
def getData(mid, gettype):
#     if mid in userdf['uid'].values:
#         return 0, []

    sleepTimes = 0
    
    fi ,fe = addUserDf(mid)
    while fi == -1:
        print('用户',str(mid),'失败')
        sleepTimes += 1
        if sleepTimes <= 3:
#             tm_mon=6, tm_mday=22, tm_hour=19, tm_min=31
            print("当前时间：",time.localtime().tm_hour," : ",time.localtime().tm_min," 休眠一次")
            time.sleep(5*60)
            fi ,fe = addUserDf(mid)
        else: return 0, []

    
    print(fi, fe)
    
    getFollowingsList(mid, fi)
    
    if gettype == 1:
        flist = getFollowersList(mid, fe)
    else : flist = []
    print('用户',str(mid),'完成')
    
    saveData()
    return 1, flist

def getDataS(li):
    for i in li:
        getData(i)

def saveData():
    userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",  encoding='utf_8_sig')
    followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",  encoding='utf_8_sig')
    
def roadData():
    userdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",index_col=0, dtype={'uid':int,'follower':int,'level':int,'coins':int,'following':int})
    followdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",index_col=0, dtype='int')
    return userdf, followdf

In [ ]:


# 创建用户信息数据表
# userdf = pd.DataFrame(columns=['uid', 'name', 'sex', 'level', 'birthday', 'coins', 'following', 'follower'])
userdf, followdf = roadData()
def main():
    # 从当前的关注数据中获取下一次遍历的用户数据  
    curlist = [27880221, 470156882, 282470852, 1531707, 13447830, 20964009, 297059545, 471925925, 245742750, 393673591, 18202105, 326499679, 512561912, 27113608, 188534271, 37916665, 316845538, 13187723, 457161443, 10277363, 176037767, 10119428, 486472919, 402576555, 14110780, 258150656, 431753564, 66391032, 500274206, 17024301, 435769810, 28797646, 20546476, 37663924, 378885845, 15982391, 456664753, 270105105, 450595813, 415479453, 280389001, 7792521, 407045223, 262453663, 386165680, 38351330, 19577966, 430969192, 12444306, 483203976, 387181687, 472747194, 216156027, 67214916, 203337614, 456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665157, 215910135, 24127511]
#     curlist = []
#     for k in [291076700]:
#         curlist += getFollowingsList(k, 300)
    
    oknum = 0
    recurlist = []
    print(curlist)
    for c in curlist:
        isok, flist = getData(c, 1)
        oknum += isok
        recurlist.extend(flist)
        
    print("完成了",str(oknum),'个人')
    
    curlist = recurlist 
    print(curlist)
    for c in curlist:
        isok, _ = getData(c, 0)
        oknum += isok
    print("完成了",str(oknum),'个人')


main()

# getData(259755161, 0)


[719238, 125526, 37961599, 18690024, 31654362, 326246517, 119801456, 27880221, 470156882, 282470852, 1531707, 13447830, 20964009, 297059545, 471925925, 245742750, 393673591, 18202105, 326499679, 512561912, 27113608, 188534271, 37916665, 316845538, 13187723, 457161443, 10277363, 176037767, 10119428, 486472919, 402576555, 14110780, 258150656, 431753564, 66391032, 500274206, 17024301, 435769810, 28797646, 20546476, 37663924, 378885845, 15982391, 456664753, 270105105, 450595813, 415479453, 280389001, 7792521, 407045223, 262453663, 386165680, 38351330, 19577966, 430969192, 12444306, 483203976, 387181687, 472747194, 216156027, 67214916, 203337614, 456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665

In [ ]:
getFollowersList

In [30]:
list(set(followdf['followid'].values))

31681

In [49]:
a = getData(35252651)
a

3 0
https://api.bilibili.com/x/relation/followings?vmid=35252651&pn=1&ps=20&order=desc&jsonp=jsonp
用户 35252651 完成


In [50]:
a

In [34]:
list(set(followdf['followid'].values))[:1000]

[1.0,
 2.0,
 32899075.0,
 21102596.0,
 3538946.0,
 4.0,
 9.0,
 17432587.0,
 12.0,
 16.0,
 18.0,
 21.0,
 3932186.0,
 454819874.0,
 275906596.0,
 131111.0,
 15466538.0,
 431751211.0,
 431358002.0,
 2621492.0,
 402260023.0,
 20054076.0,
 86900798.0,
 25821247.0,
 917568.0,
 9175105.0,
 75.0,
 39583819.0,
 353239115.0,
 131152.0,
 16777299.0,
 23461978.0,
 5898335.0,
 95.0,
 1835105.0,
 37093473.0,
 106.0,
 917612.0,
 109.0,
 44826743.0,
 384565371.0,
 917629.0,
 126.0,
 2752642.0,
 2883718.0,
 15990919.0,
 429392011.0,
 11272335.0,
 524432.0,
 38404243.0,
 155.0,
 519962788.0,
 383516841.0,
 281804983.0,
 383647930.0,
 6815935.0,
 543293632.0,
 17301696.0,
 2883778.0,
 7471298.0,
 23068871.0,
 3670216.0,
 19792072.0,
 62783699.0,
 32506073.0,
 393436.0,
 298582251.0,
 917741.0,
 6422770.0,
 328466676.0,
 24772853.0,
 24903925.0,
 267780343.0,
 33947896.0,
 131328.0,
 451150087.0,
 29360395.0,
 116654352.0,
 317718803.0,
 26870041.0,
 302121242.0,
 50331931.0,
 482083106.0,
 321782050.0,
 

### 数据分析部分

In [ ]:
def recur(curlist, recurNum):
    # 递归三次
    print('当前遍历数目', recurNum)
    if recurNum > 3:
        return 
    
    # 新的递归列表
    recurlist = []
    
    print('当前遍历列表数目', len(curlist))
    for c in curlist:
        getData(c)
    
    recurlist = list(set(recurlist))
    
    userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf"+str(recurNum)+".csv",  encoding='utf_8_sig')
    followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf"+str(recurNum)+".csv",  encoding='utf_8_sig')
    
#     recur(recurlist, recurNum+1)

In [10]:
userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf.csv",  encoding='utf_8_sig')
followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf.csv",  encoding='utf_8_sig')

In [23]:
fodf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf.csv",index_col=0)
userdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf.csv",index_col=0)

In [25]:
userdf.groupby(by='uid').size()

uid
2            1
9            1
12           1
18           1
75           1
            ..
591867189    1
591889353    1
593660504    1
596324576    1
605205509    1
Length: 3068, dtype: int64

In [17]:
saveData(userdf, 'userdf123')

userdf123_2020-06-10_12:55.csv


In [8]:
followdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90097 entries, 0 to 90096
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       90097 non-null  object
 1   followid  90097 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


In [9]:
userdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3068 entries, 0 to 3067
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uid        3068 non-null   object
 1   name       3068 non-null   object
 2   sex        3068 non-null   object
 3   level      3068 non-null   object
 4   birthday   3068 non-null   object
 5   coins      3068 non-null   object
 6   following  3068 non-null   object
 7   follower   3068 non-null   object
dtypes: object(8)
memory usage: 295.7+ KB


In [6]:
list(set(followdf['followid'].values))

{'20055735',
 '22745573',
 '471434619',
 '381816515',
 '168064909',
 '232472043',
 '297153101',
 '4249401',
 '388048178',
 '13793160',
 '43646124',
 '83375150',
 '3811052',
 '72209046',
 '684787',
 '61260',
 '336666099',
 '19577966',
 '108239596',
 '391871660',
 '386165680',
 '88884509',
 '10680360',
 '12894311',
 '4012943',
 '473940554',
 '2609425',
 '3379951',
 '393919380',
 '515993',
 '165463',
 '44412187',
 '32786875',
 '89668800',
 '10751960',
 '27439528',
 '288239',
 '104171532',
 '399194089',
 '401366826',
 '1815938',
 '331195689',
 '1424972',
 '8993755',
 '488898267',
 '16004678',
 '7061545',
 '213725662',
 '301310292',
 '491811245',
 '8014831',
 '103120101',
 '245627923',
 '121225592',
 '947124',
 '8679425',
 '2654670',
 '80900494',
 '362038251',
 '562197',
 '474521380',
 '388560897',
 '306455387',
 '2455803',
 '393673591',
 '351477766',
 '394864078',
 '38640369',
 '408940191',
 '4365978',
 '326976095',
 '220120391',
 '203337614',
 '24478681',
 '1828737',
 '7621004',
 '2249298

In [2]:
print(range(-1))

range(0, -1)


In [130]:
recur:recurlist

NameError: name 'recurlist' is not defined

In [131]:
%whos

Variable            Type         Data/Info
------------------------------------------
Get                 function     <function Get at 0x000002152EC52D90>
a                   list         n=4
addDf               function     <function addDf at 0x0000021536F87D08>
addUserDf           function     <function addUserDf at 0x000002152C23D400>
apiurl              str          https://api.bilibili.com/x/space/upstat?mid=
apiurl2             str          https://api.bilibili.com/x/space/stat?mid=
cookies             dict         n=1
followdf            DataFrame    Empty DataFrame\nColumns:<...>uid, followid]\nIndex: []
getFfNum            function     <function getFfNum at 0x0000021536FAB268>
getFollowersList    function     <function getFollowersList at 0x0000021536F2AE18>
getFollowingsList   function     <function getFollowingsLi<...>st at 0x000002152ECAE0D0>
grequests           module       <module 'grequests' from <...>-packages\\grequests.py'>
headers             dict         n=2
i     

D:\Anaconda3_5\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [123]:
userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf.csv",  encoding='utf_8_sig')
followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf.csv",  encoding='utf_8_sig')

In [112]:
a = [1,2]
a.extend([3,4])
a

[1, 2, 3, 4]

In [103]:
l = getFollowingsList(3003362, 74)
l

https://api.bilibili.com/x/relation/followings?vmid=3003362&pn=1&ps=20&order=desc&jsonp=jsonp
https://api.bilibili.com/x/relation/followings?vmid=3003362&pn=2&ps=20&order=desc&jsonp=jsonp
https://api.bilibili.com/x/relation/followings?vmid=3003362&pn=3&ps=20&order=desc&jsonp=jsonp
https://api.bilibili.com/x/relation/followings?vmid=3003362&pn=4&ps=20&order=desc&jsonp=jsonp


[10330740,
 178029850,
 238219592,
 216156027,
 18202105,
 587840959,
 483311105,
 495134012,
 435769810,
 1531707,
 404880252,
 14804670,
 33523307,
 525971445,
 517327498,
 52190805,
 386165680,
 5970160,
 37663924,
 176037767,
 6302909,
 27113608,
 25706078,
 470156882,
 27880221,
 119801456,
 282470852,
 16419172,
 406605731,
 5294454,
 23534705,
 28266043,
 3353026,
 275016013,
 258150656,
 405207322,
 14110780,
 326246517,
 291076700,
 31654362,
 326081112,
 38351330,
 254463269,
 562197,
 472747194,
 43976732,
 18690024,
 1577804,
 414878461,
 59905809,
 37961599,
 67141499,
 125526,
 21950148,
 63231,
 86439234,
 11336264,
 26798384,
 20166755,
 262453663,
 88461692,
 25876945,
 163637592,
 402827535,
 4345670,
 223146252,
 32786875,
 946974,
 29959830,
 17546432,
 17638509,
 927587,
 367877,
 7349]

In [104]:
for i in l:
    addUserDf(i)

In [108]:
userdf

10330740 in userdf['uid'].values

True

In [99]:
# 用户信息
r = requests.get("https://api.bilibili.com/x/space/acc/info?mid=3003362&jsonp=jsonp", headers=headers, cookies=cookies)
r.json()['data']

{'mid': 3003362,
 'name': '机智的小荣',
 'sex': '男',
 'face': 'http://i1.hdslb.com/bfs/face/7534b25251ba32c273008dc469b737f57af575c5.jpg',
 'sign': '',
 'rank': 10000,
 'level': 5,
 'jointime': 0,
 'moral': 0,
 'silence': 0,
 'birthday': '12-10',
 'coins': 235.4,
 'fans_badge': False,
 'official': {'role': 0, 'title': '', 'desc': '', 'type': -1},
 'vip': {'type': 2, 'status': 1, 'theme_type': 0},
 'pendant': {'pid': 0,
  'name': '',
  'image': '',
  'expire': 0,
  'image_enhance': ''},
 'nameplate': {'nid': 0,
  'name': '',
  'image': '',
  'image_small': '',
  'level': '',
  'condition': ''},
 'is_followed': False,
 'top_photo': 'http://i1.hdslb.com/bfs/space/cb1c3ef50e22b6096fde67febe863494caefebad.png',
 'theme': {},
 'sys_notice': {}}

In [97]:
followdf

,uid,followid
0,3003362,10330740
1,3003362,178029850
2,3003362,238219592
3,3003362,216156027
4,3003362,18202105
...,...,...
69,3003362,17546432
70,3003362,17638509
71,3003362,927587
72,3003362,367877


D:\Anaconda3_5\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [55]:
# 用户关注粉丝数信息
r = requests.get("https://api.bilibili.com/x/relation/stat?vmid=3003362&jsonp=jsonp", headers=headers, cookies=cookies)
r.text

'{"code":0,"message":"0","ttl":1,"data":{"mid":3003362,"following":74,"whisper":0,"black":0,"follower":2}}'

In [94]:
# 获取用户关注列表 pn页码20一页， ps每页显示最多50
r = requests.get("https://api.bilibili.com/x/relation/followings?vmid=3003362&pn=1&ps=20&order=desc&jsonp=jsonp", headers=headers, cookies=cookies)
r.json()['data']['list']

[{'mid': 10330740,
  'attribute': 2,
  'mtime': 1591633218,
  'tag': None,
  'special': 0,
  'uname': '观察者网',
  'face': 'http://i1.hdslb.com/bfs/face/1d6f100b921c9b48914eda8f3b37e76ba9ef4ca5.jpg',
  'sign': '广受年轻人心疼的时政网站（www.guancha.cn），合作联系shipin@guancha.cn。',
  'official_verify': {'type': 1, 'desc': '观察者网 官方账号'},
  'vip': {'vipType': 2,
   'vipDueDate': 1622649600000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 1,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},
 {'mid': 178029850,
  'attribute': 2,
  'mtime': 1591620417,
  'tag': None,
  'special': 0,
  'uname': '海皇Swell',
  'face': 'http://i1.hdslb.com/bfs/face/133111338ef9d6ca04515a0b517b2e837bc41e2b.jpg',
  'sign': '微博@海皇Swell 商务合作请联系QQ2284771898',
  'official_verify': {'type': 0, 'desc': 'bilibili 知名搞笑UP主'},
  'vip': {'vipType': 2,
   'vipDueDate': 1611158400000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 1,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},


In [84]:
r = requests.get("https://api.bilibili.com/x/relation/followers?vmid=3003362&pn=1&ps=20&order=desc&jsonp=jsonp", headers=headers, cookies=cookies)
r.json()['data']['list']


[{'mid': 455351753,
  'attribute': 0,
  'mtime': 1575234538,
  'tag': None,
  'special': 0,
  'uname': '薄荷本心凉q39m',
  'face': 'http://i0.hdslb.com/bfs/face/2d945b77d3989e84e6c89cb5834e898a665c0877.jpg',
  'sign': '普天之下，莫非岑土！不离不弃，并肩前行！',
  'official_verify': {'type': -1, 'desc': ''},
  'vip': {'vipType': 0,
   'vipDueDate': 0,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 0,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},
 {'mid': 27995317,
  'attribute': 0,
  'mtime': 1566404494,
  'tag': None,
  'special': 0,
  'uname': '纯粹的执着',
  'face': 'http://i0.hdslb.com/bfs/face/member/noface.jpg',
  'sign': '这个人懒死了',
  'official_verify': {'type': -1, 'desc': ''},
  'vip': {'vipType': 1,
   'vipDueDate': 1592841600000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 1,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}}]

In [44]:
# bilibili 使用动态加载，应该使用异步 grequests 库
import grequests

D:\Anaconda3_5\lib\site-packages\grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016
  curious_george.patch_all(thread=False, select=False)
D:\Anaconda3_5\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [52]:
req_list = [  # 请求列表
    grequests.get("https://space.bilibili.com/3003362/fans/follow"),
#   grequests.post('http://httpbin.org/post', data={'a':1,'b':2}),
#   grequests.put('http://httpbin.org/post', json={'a': 1, 'b': 2}),
]
res_list = grequests.map(req_list)  # 并行发送，等最后一个运行完后返回
print(res_list[0].text)


<!DOCTYPE html><html><head><meta name="spm_prefix" content="333.999"><meta charset="UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta name="renderer" content="webkit|ie-comp|ie-stand"><script type="text/javascript">window.__BILI_CONFIG__={"show_bv":true}</script><script type="text/javascript">var ua=window.navigator.userAgent,agents=["Android","iPhone","SymbianOS","Windows Phone","iPod"],pathname=/\d+/.exec(window.location.pathname),getCookie=function(e){return decodeURIComponent(document.cookie.replace(new RegExp("(?:(?:^|.*;)\\s*"+encodeURIComponent(e).replace(/[\-\.\+\*]/g,"\\$&")+"\\s*\\=\\s*([^;]*).*$)|^.*$"),"$1"))||null},DedeUserID=getCookie("DedeUserID"),mid=pathname?+pathname[0]:null===DedeUserID?0:+DedeUserID;if(mid<1)window.location.href="https://passport.bilibili.com/login?gourl=https://space.bilibili.com";else{window._bili_space_mid=mid,window._bili_space_mymid=null===DedeUserID?0:+DedeUserID;for(var prefix=/^\/v/.test(pathname)?"/v":"",i=0;i<agent

In [41]:
import requests
headers = {
#     'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'Referer':"https://www.bilibili.com/"
}
cookies = {
    "SESSDATA": "0ccb2e9a%2C1601886871%2C7146f*41"
}

In [60]:

r = requests.get("https://api.bilibili.com/x/space/navnum?mid=3003362&jsonp=jsonp&callback=__jp8", headers=headers, cookies=cookies)
r.text

'__jp8({"code":0,"message":"0","ttl":1,"data":{"video":1,"bangumi":67,"cinema":16,"channel":{"master":0,"guest":0},"favourite":{"master":10,"guest":10},"tag":0,"article":0,"playlist":0,"album":2,"audio":0,"pugv":0}})'

In [57]:
r = requests.get("https://api.bilibili.com/x/msgfeed/unread?build=0&mobi_app=web", headers=headers, cookies=cookies)
r.text

'{"code":0,"message":"0","ttl":1,"data":{"at":0,"chat":0,"like":0,"reply":0,"sys_msg":1,"up":0}}'

In [59]:
r = requests.get("https://api.bilibili.com/x/web-interface/nav?build=0&mobi_app=web", headers=headers, cookies=cookies)
r.text

'{"code":0,"message":"0","ttl":1,"data":{"isLogin":true,"email_verified":1,"face":"http://i0.hdslb.com/bfs/face/7534b25251ba32c273008dc469b737f57af575c5.jpg","level_info":{"current_level":5,"current_min":10800,"current_exp":22871,"next_exp":28800},"mid":3003362,"mobile_verified":1,"money":235.4,"moral":70,"official":{"role":0,"title":"","desc":"","type":-1},"officialVerify":{"type":-1,"desc":""},"pendant":{"pid":0,"name":"","image":"","expire":0,"image_enhance":""},"scores":0,"uname":"机智的小荣","vipDueDate":1644336000000,"vipStatus":1,"vipType":2,"vip_pay_type":0,"vip_theme_type":0,"wallet":{"mid":3003362,"bcoin_balance":0,"coupon_balance":0,"coupon_due_time":0},"has_shop":false,"shop_url":"","allowance_count":0,"answer_status":0}}'

In [ ]:
r = requests.get("https://api.bilibili.com/x/msgfeed/unread?build=0&mobi_app=web", headers=headers, cookies=cookies)
r.text

In [56]:
r = requests.get("https://api.vc.bilibili.com/session_svr/v1/session_svr/single_unread?unread_type=0&build=0&mobi_app=web", headers=headers, cookies=cookies)
r.text

'{"code":0,"msg":"ok","message":"ok","data":{"unfollow_unread":1,"follow_unread":0,"_gt_":0}}'

In [32]:
apiurl = 'https://api.bilibili.com/x/space/upstat?mid='
apiurl2 = 'https://api.bilibili.com/x/space/stat?mid='
r = requests.get(apiurl+"3003362&jsonp=jsonp", headers=headers)
r.text

'{"code":0,"message":"0","ttl":1,"data":{"archive":{"view":172},"article":{"view":0},"likes":3}}'

In [16]:
r

<Response [403]>